
## Description
---
you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the data amount. Consequently, a training sample is a tuple of 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training a end-to-end neural network from scratch whose computation is prohibitively expensive for CPUs. We divide this into two steps: feature extraction and modelling. 

- **Feature extraction**. Use the pretrained VGG network to extract features from each frame. Specifically, we recommend  to use the activations of the first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    **hints**: 
    - use `scipy.io.savemat()` to save feature to '.mat' file and `scipy.io.loadmat()` load feature.
    - norm your images using `torchvision.transforms`
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    
    ```
    More detils of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **Modelling**. With the extracted features, build an LSTM network which takes a 4096x25 sample as input, and outputs the action label of that sample.
- **Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy. Moreover, you need to compare the result of your network with that of support vector machine (SVM) (stacking the 4096x25 feature matrix to a long vector and train a SVM).

Notice that the size of the raw images is 256x340, whereas VGG16 takes 224x224 images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five 224x224 images at the image center and four corners and compute the 4096-dim VGG16 features for each of them, and average these five 4096-dim feature to get final feature representation for the raw image.

In order to save you computational time, we did the feature extraction of most samples for you except for class 1. For class 1, we provide you with the raw images, and you need to write code to extract the feature of the samples in class 1. Instead of training over the whole dataset on CPUs which mays cost you serval days, **use the first 15** classes of the whole dataset. The same applies to those who have access to GPUs.


## Dataset
Download dataset at [UCF101]

The dataset is consist of the following two parts: video images and extracted features.

### 1. Video Images  

UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images_class1/`  
  + contains videos belonging to class 1 (`ApplyEyeMakeup`)  
  + each video folder contains 25 frames  


### 2. Video Features

+ `extract_vgg16_relu6.py`  
  + used to extract video features  
     + Given an image (size: 256x340), we get 5 crops (size: 224x224) at the image center and four corners. The `vgg16-relu6` features are extracted for all 5 crops and subsequently averaged to form a single feature vector (size: 4096).  
     + Given a video, we process its 25 images seuqentially. In the end, each video is represented as a feature sequence (size: 4096 x 25).  
  + written in PyTorch; supports both CPU and GPU.  

+ `vgg16_relu6/`  
   + contains all the video features, EXCEPT those belonging to class 1 (`ApplyEyeMakeup`)  
   + you need to run script `extract_vgg16_relu6.py` to complete the feature extracting process   


## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

## 1. Feature extraction

Feature extraction is done using a VGG-16 model in a seperate file
### File name - extract_vgg16_relu6.py
The file is submitted seperately.
File is extracting features for all the movies images from folder v_00001 to v_00145 and stored the dump files in vgg16_relu6 folder. Shape of data dumped - 25*4096

## 2. Modeling


In [32]:
# write your codes here
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 

import torch
import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
from torchvision import datasets
import time
import os
import scipy.io as sio
import torch.optim as optim
import torch.utils.data as data_utils


def load_video_labels():
    label_map = {}
    type_map = {}
    file = open('UCF101_release/annos/videos_labels_subsets.txt', 'r') 
    for line in file: 
        part = line.split()
        #print(part)
        label_map[part[0]] = int(part[1])
        type_map[part[0]] = int(part[2])
    #print(label_map)
    #print(type_map)
    return label_map, type_map

# Train and evaluate
#class_1_features = get_features_from_vgg(model_updated, dataloaders_train)

#print("features done, size - ", len(class_1_features))

def load_ucf_data(dtype):
    data_list = []
    label_list = []
    label_map, type_map = load_video_labels()
    folder = os.listdir('UCF101_release/vgg16_relu6')
    try:
        folder. remove('.DS_Store')
    except:
        pass
    i = 1
    for files in folder:
        #print(files)
        name = files.split('.')
        if type_map[name[0]] != dtype or label_map[name[0]] > 15:
            continue
        data = sio.loadmat('UCF101_release/vgg16_relu6'+'/'+files)
        data = data['Feature']
        data_list.append(data)
        label_list.append(label_map[name[0]])
        #print("data set - ", i, type_map[name[0]], label_map[name[0]], files)
        i = i + 1
        #print(data['Feature'].shape)
        #break
    #return zip(data_list, label_list)
    return data_list, label_list

# Device configuration
if torch.cuda.is_available():
    print("GPU is active")
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


train_data, train_label = load_ucf_data(1)   # 1 - train 2 - test
test_data, test_label = load_ucf_data(2)
ten_train_data = data_utils.TensorDataset(torch.FloatTensor(train_data), torch.LongTensor(train_label))
tensor_train_load = data_utils.DataLoader(dataset=ten_train_data, batch_size=5, shuffle=False)

ten_test_data = data_utils.TensorDataset(torch.FloatTensor(test_data), torch.LongTensor(test_label))
tensor_test_load = data_utils.DataLoader(dataset=ten_test_data, batch_size=5, shuffle=False)

print("Length of training data - ", len(train_data))
print("Length of testing data - ", len(test_data))
#print(ten_train_data.shape)
print("Length of training data after batching - ", len(tensor_train_load))
print("Length of testing data after batching - ", len(tensor_test_load))


GPU is active
Length of training data -  1442
Length of testing data -  568
Length of training data after batching -  289
Length of testing data after batching -  114


In [27]:

class RNN(nn.Module):
    def __init__(self, inp_size, hid_size):
        super(RNN, self).__init__()
        self.hid_size = hid_size
        self.lstm = nn.LSTM(inp_size, hid_size, 2, batch_first=True)
        self.last = nn.Linear(hid_size, 15)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        init_state_1 = torch.zeros(2, x.size(0), self.hid_size).to(device) 
        init_state_2 = torch.zeros(2, x.size(0), self.hid_size).to(device)
        
        # Forward propagate LSTM
        data, _ = self.lstm(x, (init_state_1, init_state_2))
        data = self.last(data[:, -1, :])
        return data

def train_model(model, train_data, input_size, criterion, optimizer):
    since = time.time()
    total_step = len(train_data)
    for epoch in range(20):
        for run,(images, labels) in enumerate(train_data):
            #print(images.shape)
            print('epoch-',epoch+1, 'out of 20, Run-', run+1, 'out of ', total_step)
            images = images.reshape(-1, 4096, input_size).to(device)
            labels = labels - 1
            labels = labels.to(device)

            #print("label -", labels)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print('Training Loss:{:.4f}'.format(loss.item()))
    time_elapsed = time.time() - since
    print('Training completed in {:.0f}h {:.0f}m {:.0f}s'.format(time_elapsed // (60*60), time_elapsed // 60, time_elapsed % 60))
    return model


model = RNN(25, 128)
model.to(device)

optimizer_ft = optim.Adam(model.parameters(), lr=0.001)
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

model = train_model(model, tensor_train_load, 25, criterion, optimizer_ft)

epoch- 1 out of 20, Run- 1 out of  289
Training Loss:2.6964
epoch- 1 out of 20, Run- 2 out of  289
Training Loss:2.7264
epoch- 1 out of 20, Run- 3 out of  289
Training Loss:2.7244
epoch- 1 out of 20, Run- 4 out of  289
Training Loss:2.7067
epoch- 1 out of 20, Run- 5 out of  289
Training Loss:2.6941
epoch- 1 out of 20, Run- 6 out of  289
Training Loss:2.7165
epoch- 1 out of 20, Run- 7 out of  289
Training Loss:2.6608
epoch- 1 out of 20, Run- 8 out of  289
Training Loss:2.6848
epoch- 1 out of 20, Run- 9 out of  289
Training Loss:2.7569
epoch- 1 out of 20, Run- 10 out of  289
Training Loss:2.6337
epoch- 1 out of 20, Run- 11 out of  289
Training Loss:2.7742
epoch- 1 out of 20, Run- 12 out of  289
Training Loss:2.6691
epoch- 1 out of 20, Run- 13 out of  289
Training Loss:2.6948
epoch- 1 out of 20, Run- 14 out of  289
Training Loss:2.6659
epoch- 1 out of 20, Run- 15 out of  289
Training Loss:2.6470
epoch- 1 out of 20, Run- 16 out of  289
Training Loss:2.6260
epoch- 1 out of 20, Run- 17 out o

Training Loss:2.0304
epoch- 1 out of 20, Run- 136 out of  289
Training Loss:2.2079
epoch- 1 out of 20, Run- 137 out of  289
Training Loss:2.4016
epoch- 1 out of 20, Run- 138 out of  289
Training Loss:3.1037
epoch- 1 out of 20, Run- 139 out of  289
Training Loss:1.6125
epoch- 1 out of 20, Run- 140 out of  289
Training Loss:2.6403
epoch- 1 out of 20, Run- 141 out of  289
Training Loss:1.7522
epoch- 1 out of 20, Run- 142 out of  289
Training Loss:1.9345
epoch- 1 out of 20, Run- 143 out of  289
Training Loss:1.8829
epoch- 1 out of 20, Run- 144 out of  289
Training Loss:2.2001
epoch- 1 out of 20, Run- 145 out of  289
Training Loss:1.9501
epoch- 1 out of 20, Run- 146 out of  289
Training Loss:2.0539
epoch- 1 out of 20, Run- 147 out of  289
Training Loss:2.2156
epoch- 1 out of 20, Run- 148 out of  289
Training Loss:1.6747
epoch- 1 out of 20, Run- 149 out of  289
Training Loss:1.5394
epoch- 1 out of 20, Run- 150 out of  289
Training Loss:2.0930
epoch- 1 out of 20, Run- 151 out of  289
Training

Training Loss:1.0929
epoch- 1 out of 20, Run- 269 out of  289
Training Loss:2.0773
epoch- 1 out of 20, Run- 270 out of  289
Training Loss:1.5062
epoch- 1 out of 20, Run- 271 out of  289
Training Loss:1.1754
epoch- 1 out of 20, Run- 272 out of  289
Training Loss:1.4906
epoch- 1 out of 20, Run- 273 out of  289
Training Loss:1.6598
epoch- 1 out of 20, Run- 274 out of  289
Training Loss:0.6790
epoch- 1 out of 20, Run- 275 out of  289
Training Loss:1.6171
epoch- 1 out of 20, Run- 276 out of  289
Training Loss:1.7076
epoch- 1 out of 20, Run- 277 out of  289
Training Loss:2.2155
epoch- 1 out of 20, Run- 278 out of  289
Training Loss:0.9820
epoch- 1 out of 20, Run- 279 out of  289
Training Loss:1.0346
epoch- 1 out of 20, Run- 280 out of  289
Training Loss:2.1281
epoch- 1 out of 20, Run- 281 out of  289
Training Loss:1.5189
epoch- 1 out of 20, Run- 282 out of  289
Training Loss:2.0539
epoch- 1 out of 20, Run- 283 out of  289
Training Loss:0.9015
epoch- 1 out of 20, Run- 284 out of  289
Training

Training Loss:0.7441
epoch- 2 out of 20, Run- 114 out of  289
Training Loss:1.5091
epoch- 2 out of 20, Run- 115 out of  289
Training Loss:1.1002
epoch- 2 out of 20, Run- 116 out of  289
Training Loss:1.0907
epoch- 2 out of 20, Run- 117 out of  289
Training Loss:0.6677
epoch- 2 out of 20, Run- 118 out of  289
Training Loss:0.8687
epoch- 2 out of 20, Run- 119 out of  289
Training Loss:0.8984
epoch- 2 out of 20, Run- 120 out of  289
Training Loss:0.7387
epoch- 2 out of 20, Run- 121 out of  289
Training Loss:0.4192
epoch- 2 out of 20, Run- 122 out of  289
Training Loss:0.9689
epoch- 2 out of 20, Run- 123 out of  289
Training Loss:1.4900
epoch- 2 out of 20, Run- 124 out of  289
Training Loss:0.5279
epoch- 2 out of 20, Run- 125 out of  289
Training Loss:1.5519
epoch- 2 out of 20, Run- 126 out of  289
Training Loss:1.5123
epoch- 2 out of 20, Run- 127 out of  289
Training Loss:0.7479
epoch- 2 out of 20, Run- 128 out of  289
Training Loss:0.6323
epoch- 2 out of 20, Run- 129 out of  289
Training

epoch- 2 out of 20, Run- 247 out of  289
Training Loss:1.2848
epoch- 2 out of 20, Run- 248 out of  289
Training Loss:0.7815
epoch- 2 out of 20, Run- 249 out of  289
Training Loss:0.2923
epoch- 2 out of 20, Run- 250 out of  289
Training Loss:0.5103
epoch- 2 out of 20, Run- 251 out of  289
Training Loss:0.8043
epoch- 2 out of 20, Run- 252 out of  289
Training Loss:1.3712
epoch- 2 out of 20, Run- 253 out of  289
Training Loss:1.0470
epoch- 2 out of 20, Run- 254 out of  289
Training Loss:0.6043
epoch- 2 out of 20, Run- 255 out of  289
Training Loss:1.7629
epoch- 2 out of 20, Run- 256 out of  289
Training Loss:0.8830
epoch- 2 out of 20, Run- 257 out of  289
Training Loss:0.4362
epoch- 2 out of 20, Run- 258 out of  289
Training Loss:0.5032
epoch- 2 out of 20, Run- 259 out of  289
Training Loss:0.2288
epoch- 2 out of 20, Run- 260 out of  289
Training Loss:0.4252
epoch- 2 out of 20, Run- 261 out of  289
Training Loss:1.4903
epoch- 2 out of 20, Run- 262 out of  289
Training Loss:0.4867
epoch- 2

...

Training Loss:0.0092
epoch- 18 out of 20, Run- 253 out of  289
Training Loss:0.0022
epoch- 18 out of 20, Run- 254 out of  289
Training Loss:0.0025
epoch- 18 out of 20, Run- 255 out of  289
Training Loss:0.0009
epoch- 18 out of 20, Run- 256 out of  289
Training Loss:0.0007
epoch- 18 out of 20, Run- 257 out of  289
Training Loss:0.0013
epoch- 18 out of 20, Run- 258 out of  289
Training Loss:0.0008
epoch- 18 out of 20, Run- 259 out of  289
Training Loss:0.0017
epoch- 18 out of 20, Run- 260 out of  289
Training Loss:0.0010
epoch- 18 out of 20, Run- 261 out of  289
Training Loss:0.0096
epoch- 18 out of 20, Run- 262 out of  289
Training Loss:0.0005
epoch- 18 out of 20, Run- 263 out of  289
Training Loss:0.0011
epoch- 18 out of 20, Run- 264 out of  289
Training Loss:0.0003
epoch- 18 out of 20, Run- 265 out of  289
Training Loss:0.0023
epoch- 18 out of 20, Run- 266 out of  289
Training Loss:0.0011
epoch- 18 out of 20, Run- 267 out of  289
Training Loss:0.0004
epoch- 18 out of 20, Run- 268 out 

Training Loss:0.0011
epoch- 19 out of 20, Run- 96 out of  289
Training Loss:0.0012
epoch- 19 out of 20, Run- 97 out of  289
Training Loss:0.0024
epoch- 19 out of 20, Run- 98 out of  289
Training Loss:0.0007
epoch- 19 out of 20, Run- 99 out of  289
Training Loss:0.0006
epoch- 19 out of 20, Run- 100 out of  289
Training Loss:0.0025
epoch- 19 out of 20, Run- 101 out of  289
Training Loss:0.0044
epoch- 19 out of 20, Run- 102 out of  289
Training Loss:0.0012
epoch- 19 out of 20, Run- 103 out of  289
Training Loss:0.0048
epoch- 19 out of 20, Run- 104 out of  289
Training Loss:0.0007
epoch- 19 out of 20, Run- 105 out of  289
Training Loss:0.0007
epoch- 19 out of 20, Run- 106 out of  289
Training Loss:0.0005
epoch- 19 out of 20, Run- 107 out of  289
Training Loss:0.0009
epoch- 19 out of 20, Run- 108 out of  289
Training Loss:0.0006
epoch- 19 out of 20, Run- 109 out of  289
Training Loss:0.0003
epoch- 19 out of 20, Run- 110 out of  289
Training Loss:0.0010
epoch- 19 out of 20, Run- 111 out of  

Training Loss:0.0003
epoch- 19 out of 20, Run- 227 out of  289
Training Loss:0.0009
epoch- 19 out of 20, Run- 228 out of  289
Training Loss:0.0008
epoch- 19 out of 20, Run- 229 out of  289
Training Loss:0.0015
epoch- 19 out of 20, Run- 230 out of  289
Training Loss:0.0011
epoch- 19 out of 20, Run- 231 out of  289
Training Loss:0.0035
epoch- 19 out of 20, Run- 232 out of  289
Training Loss:0.0009
epoch- 19 out of 20, Run- 233 out of  289
Training Loss:0.0011
epoch- 19 out of 20, Run- 234 out of  289
Training Loss:0.0007
epoch- 19 out of 20, Run- 235 out of  289
Training Loss:0.0002
epoch- 19 out of 20, Run- 236 out of  289
Training Loss:0.0044
epoch- 19 out of 20, Run- 237 out of  289
Training Loss:0.0005
epoch- 19 out of 20, Run- 238 out of  289
Training Loss:0.0022
epoch- 19 out of 20, Run- 239 out of  289
Training Loss:0.0248
epoch- 19 out of 20, Run- 240 out of  289
Training Loss:0.0005
epoch- 19 out of 20, Run- 241 out of  289
Training Loss:0.0007
epoch- 19 out of 20, Run- 242 out 

Training Loss:0.0005
epoch- 20 out of 20, Run- 70 out of  289
Training Loss:0.0012
epoch- 20 out of 20, Run- 71 out of  289
Training Loss:0.0043
epoch- 20 out of 20, Run- 72 out of  289
Training Loss:0.0003
epoch- 20 out of 20, Run- 73 out of  289
Training Loss:0.0007
epoch- 20 out of 20, Run- 74 out of  289
Training Loss:0.0013
epoch- 20 out of 20, Run- 75 out of  289
Training Loss:0.0005
epoch- 20 out of 20, Run- 76 out of  289
Training Loss:0.0003
epoch- 20 out of 20, Run- 77 out of  289
Training Loss:0.0006
epoch- 20 out of 20, Run- 78 out of  289
Training Loss:0.0015
epoch- 20 out of 20, Run- 79 out of  289
Training Loss:0.0009
epoch- 20 out of 20, Run- 80 out of  289
Training Loss:0.0077
epoch- 20 out of 20, Run- 81 out of  289
Training Loss:0.0004
epoch- 20 out of 20, Run- 82 out of  289
Training Loss:0.0004
epoch- 20 out of 20, Run- 83 out of  289
Training Loss:0.0006
epoch- 20 out of 20, Run- 84 out of  289
Training Loss:0.0004
epoch- 20 out of 20, Run- 85 out of  289
Training

Training Loss:0.0007
epoch- 20 out of 20, Run- 201 out of  289
Training Loss:0.0028
epoch- 20 out of 20, Run- 202 out of  289
Training Loss:0.0019
epoch- 20 out of 20, Run- 203 out of  289
Training Loss:0.0007
epoch- 20 out of 20, Run- 204 out of  289
Training Loss:0.0006
epoch- 20 out of 20, Run- 205 out of  289
Training Loss:0.0008
epoch- 20 out of 20, Run- 206 out of  289
Training Loss:0.0013
epoch- 20 out of 20, Run- 207 out of  289
Training Loss:0.0006
epoch- 20 out of 20, Run- 208 out of  289
Training Loss:0.0002
epoch- 20 out of 20, Run- 209 out of  289
Training Loss:0.0009
epoch- 20 out of 20, Run- 210 out of  289
Training Loss:0.0009
epoch- 20 out of 20, Run- 211 out of  289
Training Loss:0.0017
epoch- 20 out of 20, Run- 212 out of  289
Training Loss:0.0011
epoch- 20 out of 20, Run- 213 out of  289
Training Loss:0.0003
epoch- 20 out of 20, Run- 214 out of  289
Training Loss:0.0015
epoch- 20 out of 20, Run- 215 out of  289
Training Loss:0.0007
epoch- 20 out of 20, Run- 216 out 

## 3. Evaluation

In [28]:
def test_model(model, test_data, input_size):
    with torch.no_grad():
        correct = 0
        total = 0
        i = 1
        for images, labels in test_data:
            images = images.reshape(-1, 4096, input_size).to(device)
            labels = labels - 1
            labels = labels.to(device)
            
            outputs = model(images)
            _, pred = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
            #print("test iteration - ", i)
            i = i + 1

        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    
test_model(model, tensor_test_load, 25)


Test Accuracy of the model on the test images: 80.45774647887323 %


In [29]:
import pickle
filename = 'model_epxx_h128_lr001.model'

fp = open(filename, 'wb')

pickle.dump(model, fp)

### SVM model

In [30]:
from sklearn.svm import LinearSVC

clf = LinearSVC()

total_step = len(train_data)
print('Getting the training data')
train_np_stack = np.empty(shape=[0,102400])
for features in train_data:
    features = features.flatten()
    train_np_stack = np.vstack((train_np_stack, features))

print('Training of SVM start')
since = time.time()
clf.fit(train_np_stack, train_label)
time_elapsed = time.time() - since
print('Training completed on SVM model in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Getting the training data
Training of SVM start
Training completed on SVM model in 2m 22s


In [31]:
print('Getting the testing data')
test_np_stack = np.empty(shape=[0,102400])
for features in test_data:
    features = features.flatten()
    test_np_stack = np.vstack((test_np_stack, features))

print('Starting testing')
pred = clf.predict(test_np_stack)

acc = sum(np.array(pred) == test_label)/float(len(test_label))
print("Accuracy of the SVM model {:.4f}%".format(acc* 100))

Getting the testing data
Starting testing
Accuracy of the SVM model 95.4225%
